# 🔹UFC Feature Engineering

## 1. Import Libraries and Setup Environment

In [25]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root, 'src'))
from utils.helpers import get_predictions, split_and_standardize

## 2. Load Data

In [26]:
# Define the path to the CSV file
file_path = os.path.join(project_root, 'data', 'processed', 'ufc_etl.csv')

# Load the CSV into a DataFrame
ufc_data = pd.read_csv(file_path)
print(f"Data successfully loaded: {ufc_data.shape[0]} rows, {ufc_data.shape[1]} columns.")

Data successfully loaded: 6541 rows, 55 columns.


## 3. Feature Engineering

- **Feature Vector Construction**: The feature vector for each fight is represented as:
  $$x = \text{features-fighter-blue} - \text{features-fighter-red}$$
- The task is framed as a binary classification problem, where the model predicts either **0** (Fighter Red wins) or **1** (Fighter Blue wins).

#### Create the target value: **0** (Fighter Red wins) or **1** (Fighter Blue wins)

In [27]:
ufc_data['label'] = ufc_data['Winner'].apply(lambda x: 1 if x == 'Blue' else 0)
ufc_data=ufc_data.drop('Winner', axis=1)
ufc_data = pd.get_dummies(ufc_data, columns=['TitleBout', 'Gender'], drop_first=True)

## 4. Numerical Data

### Data Split and Standarization 

In [ ]:
ufc_train, ufc_test = split_and_standardize(ufc_data)

### Final Numerical Training Data

In [ ]:
display(ufc_train.head())

### Final Numerical Testing Data

In [ ]:
display(ufc_test.head())

## 5. Categorical Data

### Stance
- **Orthodox Stance:** A fighter in orthodox stance leads with their left foot and left hand, making it the natural stance for right-handed individuals. 
- **Southpaw Stance:** A fighter in southpaw stance leads with their right foot and right hand, making it the natural stance for left-handed individuals. 
- **Open Stance Matchup:** When one fighter is orthodox and the other is southpaw, it creates an "open stance" matchup. This differs from a "closed stance" where both fighters are in the same stance (e.g., both orthodox or both southpaw)
- **Switch:** When a fighter switches from an orthodox to a southpaw stance or vice versa, it can disrupt their opponent's rhythm, causing them to miss or react incorrectly to strikes. 

In [28]:
ufc_data['BlueStance'].unique()

array(['Southpaw', 'Orthodox', 'Switch', 'Open Stance'], dtype=object)

In [29]:
# Para BlueStance
ufc_data['BlueStance_Orthodox'] = (ufc_data['BlueStance'] == 'Orthodox').astype(int)
ufc_data['BlueStance_Southpaw'] = (ufc_data['BlueStance'] == 'Southpaw').astype(int)
ufc_data['BlueStance_Switch'] = (ufc_data['BlueStance'] == 'Switch').astype(int)
ufc_data['BlueStance_OpenStance'] = (ufc_data['BlueStance'] == 'Open Stance').astype(int)

# Para RedStance (mismo enfoque)
ufc_data['RedStance_Orthodox'] = (ufc_data['RedStance'] == 'Orthodox').astype(int)
ufc_data['RedStance_Southpaw'] = (ufc_data['RedStance'] == 'Southpaw').astype(int)
ufc_data['RedStance_Switch'] = (ufc_data['RedStance'] == 'Switch').astype(int)
ufc_data['RedStance_OpenStance'] = (ufc_data['RedStance'] == 'Open Stance').astype(int)

#Borramos las columnas
ufc_data=ufc_data.drop(['BlueStance','RedStance'], axis=1)


### Better Rank

In [30]:
display(ufc_data['BetterRank'].unique())

array(['Red', 'neither', 'Blue'], dtype=object)

In [31]:
# Para BlueStance
ufc_data['BetterRank_Red'] = (ufc_data['BetterRank'] == 'Red').astype(int)
ufc_data['BetterRank_Blue'] = (ufc_data['BetterRank'] == 'Blue').astype(int)
ufc_data['BetterRank_neither'] = (ufc_data['BetterRank'] == 'neither').astype(int)
#Borramos las columnas
ufc_data=ufc_data.drop(['BetterRank'], axis=1)

## Creación de features

In [32]:
#Tasa de Finalización (Red y Blue):
RedFinishRate = (ufc_data['RedWinsByKO'] + ufc_data['RedWinsBySubmission'] + ufc_data['RedWinsByTKODoctorStoppage']) / ufc_data['RedWins'].replace(0, 1)
BlueFinishRate = (ufc_data['BlueWinsByKO'] + ufc_data['BlueWinsBySubmission'] + ufc_data['BlueWinsByTKODoctorStoppage']) / ufc_data['BlueWins'].replace(0, 1)
ufc_data['FinishRate'] = BlueFinishRate - RedFinishRate 

In [33]:
#Win Ratio (Red y Blue)
RedWinRatio = ufc_data['RedWins'] / (ufc_data['RedWins'] + ufc_data['RedLosses']).replace(0, 1)
BlueWinRatio = ufc_data['BlueWins'] / (ufc_data['BlueWins'] + ufc_data['BlueLosses']).replace(0, 1)
ufc_data['WinRatio']= BlueWinRatio - RedWinRatio 

In [34]:
#Edad vs. Experiencia
RedExpPerYear = ufc_data['RedTotalRoundsFought'] / ufc_data['RedAge']
BlueExpPerYear = ufc_data['BlueTotalRoundsFought'] / ufc_data['BlueAge']
ufc_data['ExpPerYear']=BlueExpPerYear - RedExpPerYear 

In [35]:
#Reach Advantage Ratio
ufc_data['ReachAdvantageRatio'] = ufc_data['RedReachCms'] / ufc_data['BlueReachCms']

In [36]:
#Height/Reach Ratio (para cada peleador)
RedHeightReachRatio = ufc_data['RedHeightCms'] / ufc_data['RedReachCms']
BlueHeightReachRatio = ufc_data['BlueHeightCms'] / ufc_data['BlueReachCms']
ufc_data['HeightReachRatio']= BlueHeightReachRatio - RedHeightReachRatio

In [37]:
BlueWinsByDecision = ufc_data[['BlueWinsByDecisionMajority', 'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous']].sum(axis=1)
RedWinsByDecision = ufc_data[['RedWinsByDecisionMajority', 'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous']].sum(axis=1)
ufc_data['WinsByDecision']=BlueWinsByDecision- RedWinsByDecision

In [38]:

BlueDecisionRate = BlueWinsByDecision / ufc_data['BlueWins'].replace(0, 1)  # Evitar división por cero
RedDecisionRate = RedWinsByDecision  / ufc_data['RedWins'].replace(0, 1)  # Evitar división por cero
ufc_data['DecisionRate']= BlueDecisionRate - RedDecisionRate



In [39]:
ufc_data['Gender_MALE'] = ufc_data['Gender_MALE'].astype(int)  # True → 1, False → 0

## Selección de features

Dado lo siguiente:
-  LoseStreakDif: BlueCurrentLoseStreak - RedCurrentLoseStreak
-  WinStreakDif: BlueCurrentWinStreak - RedCurrentWinStreak
-  LongestWinStreakDif: BlueLongestWinStreak - 'RedLongestWinStreak
-  WinDif: BlueWins - RedWins
-  LossDif: BlueLosses - RedLosses
-  TotalRoundDif: BlueTotalRoundsFought - RedTotalRoundsFought
-  TotalTitleBoutDif: BlueTotalTitleBouts - RedTotalTitleBouts
-  KODif: BlueWinsByKO - RedWinsByKO
-  SubDif: BlueWinsBySubmission - RedWinsBySubmission
-  HeightDif: BlueHeightCms - RedHeightCms
-  ReachDif: BlueReachCms - RedReachCms
-  AgeDif: BlueAge - RedAge
  
Se eliminan ciertas columnas que pueden ser redundantes.

In [40]:
ufc_data=ufc_data.drop(['BlueCurrentLoseStreak', 'RedCurrentLoseStreak','BlueCurrentWinStreak',
                      'RedCurrentWinStreak','BlueLongestWinStreak', 'RedLongestWinStreak', 'BlueWins',
                      'RedWins','BlueLosses','RedLosses', 'BlueTotalRoundsFought','RedTotalRoundsFought',
                      'BlueTotalTitleBouts', 'RedTotalTitleBouts', 'BlueWinsByKO', 'RedWinsByKO', 'BlueWinsBySubmission',
                      'RedWinsBySubmission','BlueHeightCms','RedHeightCms','BlueReachCms','RedReachCms',
                      'BlueAge', 'RedAge'], axis=1)

También aquellas relacionadas con los features construidos:
- WinsByDecision
- DecisionRate
- FinishRate


In [41]:
ufc_data=ufc_data.drop(['BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous',
       'BlueWinsByTKODoctorStoppage', 'RedWinsByDecisionMajority',
       'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous',
       'RedWinsByTKODoctorStoppage','BlueWinsByDecisionMajority'], axis=1)

### Columnas de baja varianza
- BlueDraws
- RedDraws
- BlueWeightLbs
- RedWeightLbs
- TitleBout_True 

In [42]:
ufc_data=ufc_data.drop(['BlueDraws','RedDraws','BlueWeightLbs','RedWeightLbs','TitleBout_True'], axis=1)

In [43]:
# Preview the first few records
display(ufc_data.head())
display(ufc_data.columns)
# Para ver los tipos de dato de cada columna:
display(ufc_data.dtypes)

NumberOfRounds  LoseStreakDif  WinStreakDif  LongestWinStreakDif  WinDif  \
0               5             -1             5                   -2      -2   
1               3              0             0                   -2     -12   
2               3             -1             4                    0       0   
3               3              1             0                   -1       3   
4               3              0             3                   -2      -3   

   LossDif  TotalRoundDif  TotalTitleBoutDif  KODif  SubDif  ...  \
0        0            -24                 -4      4      -2  ...   
1       -9            -54                  0     -2      -1  ...   
2       -1             -1                  0      1       1  ...   
3        5             21                  0      2      -1  ...   
4       -2             -2                  0     -5       0  ...   

   BetterRank_Red  BetterRank_Blue  BetterRank_neither  FinishRate  WinRatio  \
0               1                0                   0    0.283333 -0.035714   
1               0                0                   1    0.293233  0.042614   
2               1                0                   0    0.500000  0.095238   
3               0                0                   1   -0.100000 -0.261905   
4               0                0                   1   -0.607143  0.222222   

   ExpPerYear  ReachAdvantageRatio  HeightReachRatio  WinsByDecision  \
0   -0.477778             0.947368         -0.065058              -4   
1   -1.222222             1.000000          0.028571              -9   
2   -0.077799             1.046154          0.028733              -2   
3    0.453704             1.013158          0.025803               2   
4   -0.064516             1.014493          0.013872               2   

   DecisionRate  
0     -0.283333  
1     -0.293233  
2     -0.500000  
3      0.100000  
4      0.607143  

[5 rows x 33 columns]

Index(['NumberOfRounds', 'LoseStreakDif', 'WinStreakDif',
       'LongestWinStreakDif', 'WinDif', 'LossDif', 'TotalRoundDif',
       'TotalTitleBoutDif', 'KODif', 'SubDif', 'HeightDif', 'ReachDif',
       'AgeDif', 'label', 'Gender_MALE', 'BlueStance_Orthodox',
       'BlueStance_Southpaw', 'BlueStance_Switch', 'BlueStance_OpenStance',
       'RedStance_Orthodox', 'RedStance_Southpaw', 'RedStance_Switch',
       'RedStance_OpenStance', 'BetterRank_Red', 'BetterRank_Blue',
       'BetterRank_neither', 'FinishRate', 'WinRatio', 'ExpPerYear',
       'ReachAdvantageRatio', 'HeightReachRatio', 'WinsByDecision',
       'DecisionRate'],
      dtype='object')

NumberOfRounds             int64
LoseStreakDif              int64
WinStreakDif               int64
LongestWinStreakDif        int64
WinDif                     int64
LossDif                    int64
TotalRoundDif              int64
TotalTitleBoutDif          int64
KODif                      int64
SubDif                     int64
HeightDif                float64
ReachDif                 float64
AgeDif                     int64
label                      int64
Gender_MALE                int64
BlueStance_Orthodox        int64
BlueStance_Southpaw        int64
BlueStance_Switch          int64
BlueStance_OpenStance      int64
RedStance_Orthodox         int64
RedStance_Southpaw         int64
RedStance_Switch           int64
RedStance_OpenStance       int64
BetterRank_Red             int64
BetterRank_Blue            int64
BetterRank_neither         int64
FinishRate               float64
WinRatio                 float64
ExpPerYear               float64
ReachAdvantageRatio      float64
HeightReac

In [44]:
display(ufc_data.head())

NumberOfRounds  LoseStreakDif  WinStreakDif  LongestWinStreakDif  WinDif  \
0               5             -1             5                   -2      -2   
1               3              0             0                   -2     -12   
2               3             -1             4                    0       0   
3               3              1             0                   -1       3   
4               3              0             3                   -2      -3   

   LossDif  TotalRoundDif  TotalTitleBoutDif  KODif  SubDif  ...  \
0        0            -24                 -4      4      -2  ...   
1       -9            -54                  0     -2      -1  ...   
2       -1             -1                  0      1       1  ...   
3        5             21                  0      2      -1  ...   
4       -2             -2                  0     -5       0  ...   

   BetterRank_Red  BetterRank_Blue  BetterRank_neither  FinishRate  WinRatio  \
0               1                0                   0    0.283333 -0.035714   
1               0                0                   1    0.293233  0.042614   
2               1                0                   0    0.500000  0.095238   
3               0                0                   1   -0.100000 -0.261905   
4               0                0                   1   -0.607143  0.222222   

   ExpPerYear  ReachAdvantageRatio  HeightReachRatio  WinsByDecision  \
0   -0.477778             0.947368         -0.065058              -4   
1   -1.222222             1.000000          0.028571              -9   
2   -0.077799             1.046154          0.028733              -2   
3    0.453704             1.013158          0.025803               2   
4   -0.064516             1.014493          0.013872               2   

   DecisionRate  
0     -0.283333  
1     -0.293233  
2     -0.500000  
3      0.100000  
4      0.607143  

[5 rows x 33 columns]

## 6. Save

In [48]:
# Save the cleaned file
ufc_data.to_csv(f'{project_root}/data/processed/ufc_processed.csv', index=False)
print("\nFeature Engineering file saved as 'ufc_processed.csv'.")


Feature Engineering file saved as 'ufc_processed.csv'.
